In [1]:
from langchain.llms import OpenAI
import os


## Using Self-Defined Agent to Query Wikipedia

In [2]:
import wikipediaapi
import requests

In [3]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
response = llm.invoke("What are the keywords to search for on Wikipedia given the claim Beijing is the capital of China. Please give the answer in a common delimited sentence with no space.")
print(response)

content='Beijing,capital,China' response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 39, 'total_tokens': 46}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None} id='run-8aa076cf-623a-4d7a-9636-1511ac11598b-0'


In [5]:
def cleanup(msg):
    msg_lst = dict(msg)['content'].split(',')
    return msg_lst

cleanup(response)


['Beijing', 'capital', 'China']

In [6]:
url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "prop": "extracts",
    "titles": 'beijing',
    "explaintext": True,
    "exintro": True,
    "explaintext": True 
}
response = requests.get(url, params=params)
data = response.json()
pages = data["query"]["pages"]
counter = 0
info = ""
for page_id in pages:
    counter += 1
    page = pages[page_id]
    info += page['extract']
info

"Beijing, alternatively romanized as Peking, is the capital of China. With about 22 million residents, Beijing is the world's most populous national capital city as well as China's second largest city after Shanghai. It is located in Northern China, and is governed as a municipality under the direct administration of the State Council with 16 urban, suburban, and rural districts. Beijing is mostly surrounded by Hebei Province with the exception of neighboring Tianjin to the southeast; together, the three divisions form the Jingjinji megalopolis and the national capital region of China.Beijing is a global city and one of the world's leading centres for culture, diplomacy, politics, finance, business and economics, education, research, language, tourism, media, sport, science and technology and transportation. It is home to the headquarters of most of China's largest state-owned companies and houses the largest number of Fortune Global 500 companies in the world, as well as the world's f

In [7]:
response = llm.invoke("Answer yes or no to the claim Bejing is the captital of China given this information: " + "Beijing is the captial of America")

In [8]:
dict(response)['content']

'No'

In [9]:
from langchain.agents import tool

@tool
def verify_claim(claim: str) -> str:
    """Returns the yes or now to verify a claim"""
    response = llm.invoke("What are the keywords to search for on Wikipedia given the claim: " + claim + "Please give the answer in a comma delimited sentence with no space.")
    def cleanup(msg):
        msg_lst = dict(msg)['content'].split(',')
        return msg_lst
    keyword = cleanup(response)[0]

    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "titles": keyword,
        "explaintext": True,
        "exintro": True,
        "explaintext": True 
    }
    response = requests.get(url, params=params)
    data = response.json()
    pages = data["query"]["pages"]
    info = ""
    for page_id in pages:
        page = pages[page_id]
        info += page['extract']

    response = llm.invoke("Verify the following claim " + claim + " Given this information: " + info)
    return dict(response)['content']


In [10]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a powerful fact checker.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [11]:
from langchain.tools.render import format_tool_to_openai_function

tools = [verify_claim]
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [12]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [13]:
from langchain.schema.agent import AgentFinish

user_input = "Beijing is the capital of China"
intermediate_steps = []

while True:
    output = agent.invoke(
        {
            "input": user_input,
            "intermediate_steps": intermediate_steps,
        }
    )
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(f"TOOL NAME: {output.tool}")
        print(f"TOOL INPUT: {output.tool_input}")
        tool = {"verify_claim": verify_claim}[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))

print(final_result)

TOOL NAME: verify_claim
TOOL INPUT: {'claim': 'Beijing is the capital of China'}
The claim that Beijing is the capital of China is verified.


In [14]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Beijing is the capital of China"})



> Entering new AgentExecutor chain...

Invoking: `verify_claim` with `{'claim': 'Beijing is the capital of China'}`


The claim that Beijing is the capital of China is verified by the information provided.The claim that Beijing is the capital of China is verified.

> Finished chain.


{'input': 'Beijing is the capital of China',
 'output': 'The claim that Beijing is the capital of China is verified.'}

## Using Predefined Agent Type: Self-Ask with Search

In [15]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.llms import Fireworks
from langchain_community.tools.tavily_search import TavilyAnswer

In [16]:
tools = [TavilyAnswer(max_results=1, name="Intermediate Answer")]

In [17]:
prompt = hub.pull("hwchase17/self-ask-with-search")

In [18]:
llm = Fireworks()
agent = create_self_ask_with_search_agent(llm, tools, prompt)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.fireworks.Fireworks` was deprecated in langchain-community 0.0.26 and will be removed in 0.2. An updated version of the class exists in the langchain-fireworks package and should be used instead. To use it run `pip install -U langchain-fireworks` and import as `from langchain_fireworks import Fireworks`.
  warn_deprecated(


In [19]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [21]:
response  = agent_executor.invoke(
    {"input": "Is the capital of China Beijing?"}
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: What is the capital of China?Beijing is the capital of China.So the final answer is: Yes.

> Finished chain.


In [22]:
final_answer = response.get("output", "")
final_answer

'Yes.'

## Using Prebuilt Agent: SQL Database Agent

In [23]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [24]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [25]:
agent_executor.invoke(
    "Verify the claim: Customers in the United States spent the most money"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks
Invoking: `sql_db_schema` with `{'table_names': 'customers, invoices, invoice_items'}`



CREATE TABLE customers (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES employees ("EmployeeId")
)

/*
3 rows from customers table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigade

{'input': 'Verify the claim: Customers in the United States spent the most money',
 'output': 'Customers in the United States did not spend the most money. The country that spent the most money is the USA with a total amount spent of $523.06.'}